# NNodely Documentation - Equation Learner Layer

Represents a nnodely implementation of the Task-Parametrized Equation Learner block.

Official Paper: [Task-Parametrized Equation Learner](https://www.sciencedirect.com/science/article/pii/S0921889022001981)

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *

## Basic Usage
Create an simple equation learner block using nnodely trigonometric functions. The initial linear layer is created using random initialization.

In [ ]:
x = Input('x')
equation_learner = EquationLearner(functions=[Tan, Sin, Cos])
out = Output('out',equation_learner(x.last()))

## Input layer
Create an simple equation learner block using nnodely trigonometric functions and passing an input layer

In [ ]:
x = Input('x')
input_layer = Linear(output_dimension=3, W_init=init_constant, W_init_params={'value':1}, b=False)
equation_learner = EquationLearner(functions=[Tan, Sin, Cos], linear_in=input_layer)
out = Output('out', equation_learner(x.last()))

## Input layer and output layer
Create an simple equation learner block using nnodely trigonometric functions and passing an input layer and also a linear output layer

(By default, there is no linear output layer)

In [ ]:
x = Input('x')
input_layer = Linear(output_dimension=3, W_init=init_constant, W_init_params={'value':1}, b=False)
output_layer = Linear(output_dimension=1, W_init=init_constant, W_init_params={'value':1}, b=True)
equation_learner = EquationLearner(functions=[Tan, Sin, Cos], linear_in=input_layer, linear_out=output_layer)
out = Output('out', equation_learner(x.last()))

## Multiple inputs
Create an simple equation learner block using nnodely trigonometric functions and passing multiple inputs when calling the equation layer block.
All the given inputs will be concatenated before going through the linear input layer

In [ ]:
x = Input('x')
F = Input('F')
equation_learner = EquationLearner(functions=[Tan, Sin, Cos])
out = Output('out',equation_learner(inputs=(x.last(),F.last())))

## Multi-parameter functions
Create an equation learner block with functions that take 2 parameters (add, sub, mul ...). 

!! Be careful to the output dimension that the linear input layer should have to connect correctly all the activation functions. In the example below, both the Add and Mul relations take 2 parameters so the total number of output dimension is 7 instead of 5

In [ ]:
x = Input('x')
F = Input('F')

linear_layer_in_1 = Linear(output_dimension=7)
equation_learner_1 = EquationLearner(functions=[Tan, Add, Sin, Mul, Identity], linear_in=linear_layer_in_1)
out = Output('out',equation_learner_1(x.last()))

## Using custom parametric functions
Create an equation learner block with simple parametric functions

In [ ]:
import torch

def func1(K1):
    return torch.sin(K1)

def func2(K2):
    return torch.cos(K2)

x = Input('x')
parfun1 = ParamFun(func1)
parfun2 = ParamFun(func2)
equation_learner = EquationLearner([parfun1, parfun2])
out = Output('out',equation_learner(x.last()))

## Using parametric functions with parameters
Create an equation learner block with simple parametric functions

In [ ]:
def myFun(K1,K2,p1,p2):
    return K1*p1+K2*p2

x = Input('x')
F = Input('F')

K1 = Parameter('k1', dimensions =  1, sw = 1,values=[[2.0]])
K2 = Parameter('k2', dimensions =  1, sw = 1,values=[[3.0]])
parfun = ParamFun(myFun, parameters_and_constants=[K1,K2])

equation_learner = EquationLearner([parfun, Sin, Add])
out = Output('out',equation_learner((x.last(),F.last())))

## Parametric functions and fuzzy layers
Create an equation learner block with parametric functions and fuzzy layers

In [ ]:
def myFun(K1,p1):
    return K1*p1

x = Input('x')
F = Input('F')

K = Parameter('k', dimensions =  1, sw = 1,values=[[2.0]])
parfun = ParamFun(myFun, parameters_and_constants = [K])

fuzzi = Fuzzify(centers=[0,1,2,3])
equation_learner = EquationLearner([parfun, fuzzi])
out = Output('out',equation_learner((x.last(),F.last())))

## Cascade Equation Learner Blocks
Create a cascade of equation learner blocks with various functions and temporal window inputs

In [ ]:
x = Input('x')
F = Input('F')

def myFun(K1,K2,p1,p2):
    return K1*p1+K2*p2

K1 = Parameter('k1', dimensions =  1, sw = 1,values=[[2.0]])
K2 = Parameter('k2', dimensions =  1, sw = 1,values=[[3.0]])
parfun = ParamFun(myFun, parameters_and_constants = [K1,K2])

input_layer_1 = Linear(output_dimension=5, W_init=init_constant, W_init_params={'value':1}, b_init=init_constant, b_init_params={'value':0})
input_layer_2 = Linear(output_dimension=7, W_init=init_constant, W_init_params={'value':1}, b_init=init_constant, b_init_params={'value':0})
output_layer = Linear(output_dimension=1, W_init=init_constant, W_init_params={'value':1}, b=True)
equation_learner = EquationLearner([parfun, Sin, Add], linear_in=input_layer_1)
equation_learner_2 = EquationLearner(functions=[Tan, Add, Sin, Mul, Identity], linear_in=input_layer_2, linear_out=output_layer)

out = Output('out',equation_learner_2(equation_learner((x.sw(1),F.sw(1)))))